In [36]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [37]:
df=pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202212.csv')
df.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,23324279,제중건강원,NaN,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,...,1156010600100410002034626,NaN,서울특별시 영등포구 영중로14길 11-17,150035,7250.0,NaN,1,NaN,126.907168,37.520613
1,24525909,민속악기사,NaN,D,소매,D04,취미/오락관련소매,D04A09,악기판매,G47593,...,1120012200101420011000227,NaN,서울특별시 성동구 용답5길 2,133849,4803.0,NaN,NaN,NaN,127.049018,37.566857
2,24715368,태평양진주,NaN,D,소매,D26,시계/귀금속소매,D26A01,시계/귀금속,G47830,...,1111015200100440000000001,금사랑투빌딩,서울특별시 종로구 서순라길 17-10,110390,3138.0,NaN,NaN,NaN,126.993530,37.571848
3,15554136,김선희꼼꼼국어교습소,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,1171011100101970003017195,NaN,서울특별시 송파구 위례성대로12길 31,138834,5640.0,NaN,2,NaN,127.121520,37.510967
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377724 entries, 0 to 377723
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     377724 non-null  int64  
 1   상호명        377724 non-null  object 
 2   지점명        63169 non-null   object 
 3   상권업종대분류코드  377724 non-null  object 
 4   상권업종대분류명   377724 non-null  object 
 5   상권업종중분류코드  377724 non-null  object 
 6   상권업종중분류명   377724 non-null  object 
 7   상권업종소분류코드  377724 non-null  object 
 8   상권업종소분류명   377724 non-null  object 
 9   표준산업분류코드   351543 non-null  object 
 10  표준산업분류명    351543 non-null  object 
 11  시도코드       377724 non-null  int64  
 12  시도명        377724 non-null  object 
 13  시군구코드      377724 non-null  int64  
 14  시군구명       377724 non-null  object 
 15  행정동코드      377724 non-null  int64  
 16  행정동명       377724 non-null  object 
 17  법정동코드      377724 non-null  int64  
 18  법정동명       377724 non-null  object 
 19  지번코드       377724 non-n

In [39]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [40]:
df.isnull().sum()

상가업소번호            0
상호명               0
지점명          314555
상권업종대분류코드         0
상권업종대분류명          0
상권업종중분류코드         0
상권업종중분류명          0
상권업종소분류코드         0
상권업종소분류명          0
표준산업분류코드      26181
표준산업분류명       26181
시도코드              0
시도명               0
시군구코드             0
시군구명              0
행정동코드             0
행정동명              0
법정동코드             0
법정동명              0
지번코드              0
대지구분코드            0
대지구분명             0
지번본번지             0
지번부번지         78602
지번주소              0
도로명코드             0
도로명               0
건물본번지             0
건물부번지        327127
건물관리번호            0
건물명          190057
도로명주소             0
구우편번호             0
신우편번호            36
동정보          333667
층정보          146271
호정보          377724
경도                0
위도                0
dtype: int64

In [41]:
df.groupby('상권업종대분류명').count()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
상권업종대분류명,,,,,,,,,,,,,,,,,,,,,
관광/여가/오락,9820,9820,1061,9820,9820,9820,9820,9820,8845,8845,...,9820,4588,9820,9820,9820,1032,7190,0,9820,9820
부동산,19784,19784,241,19784,19784,19784,19784,19784,17880,17880,...,19784,10869,19784,19784,19779,2567,12106,0,19784,19784
생활서비스,79679,79679,5376,79679,79679,79679,79679,79679,61833,61833,...,79679,40702,79679,79679,79669,11988,44932,0,79679,79679
소매,107460,107460,19149,107460,107460,107460,107460,107460,103198,103198,...,107460,58083,107460,107460,107452,16364,61153,0,107460,107460
숙박,2502,2502,108,2502,2502,2502,2502,2502,1610,1610,...,2502,1052,2502,2502,2501,88,1330,0,2502,2502
스포츠,5420,5420,561,5420,5420,5420,5420,5420,5420,5420,...,5420,3554,5420,5420,5420,703,3618,0,5420,5420
음식,125771,125771,33166,125771,125771,125771,125771,125771,125469,125469,...,125771,51615,125771,125771,125760,6494,83920,0,125771,125771
학문/교육,27288,27288,3507,27288,27288,27288,27288,27288,27288,27288,...,27288,17204,27288,27288,27287,4821,17204,0,27288,27288


In [42]:
df_food = df.loc[df.상권업종대분류명 == '음식']
df_food.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
5,17174119,쓰리에프,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1159010700101390073009536,NaN,서울특별시 동작구 동작대로27가길 12,156816,7008.0,NaN,NaN,NaN,126.980952,37.487105
6,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010800114850004022127,NaN,서울특별시 서초구 효령로 230,137869,6709.0,NaN,1,NaN,127.009382,37.483436
7,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,민속주점,I56219,...,1168010700106150001009703,코끼리상가,서울특별시 강남구 압구정로 216,135894,6023.0,NaN,1,NaN,127.031012,37.528073
12,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1174010200102170000018014,고덕그라시움,서울특별시 강동구 고덕로 353,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197


In [43]:
df_f = df_food[['상호명','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류명','시군구명','행정동명','도로명주소']]
df_f.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시군구명,행정동명,도로명주소
4,비지트,음식,한식,한식/백반/한정식,한식 음식점업,서초구,방배4동,서울특별시 서초구 동광로18길 82
5,쓰리에프,음식,한식,한식/백반/한정식,한식 음식점업,동작구,사당2동,서울특별시 동작구 동작대로27가길 12
6,다향,음식,한식,한식/백반/한정식,한식 음식점업,서초구,서초3동,서울특별시 서초구 효령로 230
7,고향생막걸리,음식,유흥주점,민속주점,기타 주점업,강남구,압구정동,서울특별시 강남구 압구정로 216
12,도전최강달인왕만두,음식,한식,한식/백반/한정식,한식 음식점업,강동구,고덕2동,서울특별시 강동구 고덕로 353


In [44]:
df_f['시군구명'].unique()

array(['서초구', '동작구', '강남구', '강동구', '마포구', '송파구', '양천구', '성동구', '종로구',
       '광진구', '중구', '은평구', '도봉구', '노원구', '관악구', '강서구', '동대문구', '강북구',
       '성북구', '서대문구', '용산구', '중랑구', '구로구', '영등포구', '금천구'], dtype=object)

In [45]:
df_f['상권업종중분류명'].unique()

array(['한식', '유흥주점', '커피점/카페', '패스트푸드', '닭/오리요리', '중식', '양식', '제과제빵떡케익',
       '분식', '별식/퓨전요리', '기타음식업', '일식/수산물', '부페', '음식배달서비스'], dtype=object)

In [46]:
df_f['상권업종소분류명'].unique()

array(['한식/백반/한정식', '민속주점', '족발/보쌈전문', '곱창/양구이전문', '커피전문점/카페/다방', '토스트전문',
       '삼계탕전문', '중국음식/중국집', '패밀리레스토랑', '제과점', '떡전문', '부대찌개/섞어찌개',
       '국수/만두/칼국수', '죽전문점', '호프/맥주', '빠/카페/스탠드빠', '도시락전문점', '해장국/감자탕',
       '두부요리전문', '파전전문', '라면김밥분식', '후라이드/양념치킨', '구내식당/자급식음식점', '패스트푸드',
       '소주방/포장마차', '갈비/삼겹살', '정통양식/경양식', '돌솥/비빕밥전문점', '복전문', '도너츠전문',
       '로바다야끼', '룸살롱/단란주점', '횟집', '음식점-일식', '스파게티전문점', '기사식당', '갈치/생선구이',
       '아구전문', '떡볶이전문', '냉면집', '아이스크림판매', '종합부페', '찜닭전문점', '꼬치구이전문점',
       '순대전문점', '보드게임카페', '기타일반유흥주점', '해물찜/탕전문', '닭갈비전문', '추어탕전문',
       '순두부전문', '스낵', '닭내장/닭발요리', '곰장어전문', '생과일주스전문점', '기타전문서양음식점',
       '떡/한과집', '피자전문', '참치전문점', '우동전문점', '설렁탕집', '전통찻집/인삼찻집', '홍어전문',
       '야식집', '오리고기전문', '한정식전문', '음식점-초밥전문', '샌드위치전문점', '민물장어전문', '바베큐전문',
       '돈가스전문점', '핫도그', '낙지/오징어', '샤브샤브전문', '기타고기요리', '매운탕전문',
       '수산물전문음식점-종합', '음식출장조달', '퓨전음식전문', '게장전문', '동남아음식전문점', '사철탕전문',
       '철판구이요리전문', '보리밥전문', '쌈밥전문', '고등어전문', '국/찌개배달', '황태전문', '스테이크전문점',
       '고기부페'

In [47]:
df_f['상권업종소분류명'].value_counts()

한식/백반/한정식      30560
커피전문점/카페/다방    20841
라면김밥분식          5986
호프/맥주           5985
정통양식/경양식        4872
               ...  
유산균아이스크림전문         2
해물부페               2
징기스칸요리전문           1
재첩국전문              1
락카페                1
Name: 상권업종소분류명, Length: 115, dtype: int64

In [48]:
df_f['표준산업분류명'].unique()

array(['한식 음식점업', '기타 주점업', '비알콜 음료점업', '피자, 햄버거, 샌드위치 및 유사 음식점업',
       '중식 음식점업', '서양식 음식점업', '제과점업', '분식 및 김밥 전문점', '일반유흥 주점업', '치킨 전문점',
       '기관구내식당업', '그외 기타 음식점업', '일식 음식점업', nan, '이동 음식업', '출장 음식 서비스업',
       '기타 외국식 음식점업'], dtype=object)

In [49]:
df_f['표준산업분류명'].value_counts()

한식 음식점업                    47943
비알콜 음료점업                   21621
분식 및 김밥 전문점                10093
기타 주점업                      8246
서양식 음식점업                    7374
제과점업                        7043
일식 음식점업                     5413
치킨 전문점                      4360
중식 음식점업                     3892
일반유흥 주점업                    3726
피자, 햄버거, 샌드위치 및 유사 음식점업     2730
그외 기타 음식점업                  1771
기타 외국식 음식점업                  838
기관구내식당업                      332
이동 음식업                        53
출장 음식 서비스업                    34
Name: 표준산업분류명, dtype: int64

In [50]:
df['상권업종대분류명'].unique()

array(['소매', '학문/교육', '음식', '부동산', '생활서비스', '관광/여가/오락', '숙박', '스포츠'],
      dtype=object)

In [51]:
df_ent = df.loc[df.상권업종대분류명 == '관광/여가/오락']
df_ent.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
35,20783574,선경바둑,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A09,기원,R91293,...,1168010700106380005009660,고려빌딩,서울특별시 강남구 압구정로 310,135896,6017.0,NaN,3,NaN,127.035280,37.528775
63,24051369,OK노래연습장,NaN,N,관광/여가/오락,N02,무도/유흥/가무,N02A01,노래방,R91223,...,1130510100100520056028420,NaN,서울특별시 강북구 월계로7나길 9,142100,1222.0,NaN,NaN,NaN,127.033349,37.612574
84,15591157,득템pc방,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A01,인터넷PC방,R91222,...,1138010400104980003031156,제영빌딩,서울특별시 은평구 서오릉로 192,122814,3335.0,NaN,지,NaN,126.912296,37.613746
88,24676601,럭셔리노래연습장,NaN,N,관광/여가/오락,N02,무도/유흥/가무,N02A01,노래방,R91223,...,1126010500104580002003605,NaN,서울특별시 중랑구 용마산로 495,131230,2173.0,3,NaN,NaN,127.099639,37.595150
106,24200314,짱노래연습장,NaN,N,관광/여가/오락,N02,무도/유흥/가무,N02A01,노래방,R91223,...,1129010100101450001000001,NaN,서울특별시 성북구 성북로14나길 27,136825,2835.0,NaN,NaN,NaN,127.002821,37.594993


In [52]:
df_ent['상권업종중분류명'].unique()

array(['PC/오락/당구/볼링등', '무도/유흥/가무', '연극/영화/극장', '요가/단전/마사지', '스포츠/운동',
       '놀이/여가/취미', '경마/경륜/성인오락'], dtype=object)